In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
X = pd.read_csv('../../master_data/nilm/kettle_aggregates_test.dat', header = None, delim_whitespace = True,
                                index_col = 0)
y = pd.read_csv('../../master_data/nilm/kettle_signatures_test.dat', header = None, delim_whitespace = True,
                                index_col = 0)

In [ ]:
normalization = pd.read_csv('../master_data/nilm/normalization_params.csv', header=0, delim_whitespace=True)
normalization.head()

In [ ]:
X_np = np.array(X).reshape((X.shape[0],X.shape[1],1))
y_np = np.array(y).reshape((y.shape[0],y.shape[1]))

#mean = normalization['mean'].values[0]
mean = X_np.mean(axis=1).reshape(X_np.shape[0],1,1)
X_np = X_np - mean
sd = normalization['sd'].values[0]
#rand_sd = rand_sd.sample(frac=1).reset_index(drop=True)
X_np /= sd
print("Mean: ", X_np.mean())
print("Std: ", X_np.std())


In [ ]:
from keras.models import load_model

model = load_model('../master_data/nilm/models/model0123dAE.h5')

In [ ]:
pred = model.predict(X_np)

In [ ]:
#first get an idea about the results:
for i in range(4):
    dice = np.random.randint(0,y_np.shape[0])
    plt.subplot(221 + i)
    plt.plot(pred[dice])
    plt.plot(y_np[dice])
    plt.show()